In [ ]:
import os
import pandas as pd
from functools import reduce
from sklearn import metrics, model_selection

In [ ]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Call Files

In [ ]:
## Enter the path of the results folders
Path = 'drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/' 
FolderNames = os.listdir(Path)  ### get folder Results Names
FolderNames.sort()  ### folder in Alphabetic 
FolderEndPath = [Path + folder for folder in FolderNames]  ### add path to each folder

print(FolderNames)

['EnglishBert_enDataTrain', 'EnglishBert_wholeDataTrain', 'MultilingualBert_wholeDataTrain', 'SpanishBert_esDataTrain', 'SpanishBert_wholeDataTrain', 'exitis2021_AI-UPV', 'exitis2021_AI-UPV (ALL)']


In [ ]:
## Add the last necessario item to the results folders the TASK ('task1' or 'task2')
FolderEndPathTask1 = [FeP + '/Task1/' for FeP in FolderEndPath]
FolderEndPathTask2 = [FeP + '/Task2/' for FeP in FolderEndPath]

In [ ]:
FolderEndPathTask1 = {model: FeP + '/Task1/' for FeP, model in zip(FolderEndPath, FolderNames)}

In [ ]:
## Criate Dictonary to saive TASK1 files results
Task1TestFiles = {}

In [ ]:
######## Multilingual Bert ############
#File test results Multilingual Bert Trained in the Whole data
Task1TestFiles['BertMutiLingual_TrainedwholeData_TestWholeData'] = FolderEndPathTask1['MultilingualBert_wholeDataTrain'] + 'ModelInfereneces_MultilingualBertTask1wholeDataTrain_ResultsBestModel_WholeSetTest_CSV_'

In [ ]:
######## English Bert ############
#File test results English Bert Trained in the Whole data
####test whole data
Task1TestFiles['BertEnglish_TrainedwholeData_TestWholeData'] = FolderEndPathTask1['EnglishBert_wholeDataTrain'] + 'ModelInfereneces_EnglishBertTask1wholeDataTrain_ResultsBestModel_WholeSetTest_CSV_'
####test English data
Task1TestFiles['BertEnglish_TrainedwholeData_TestEnglishData'] = FolderEndPathTask1['EnglishBert_wholeDataTrain'] + 'ModelInfereneces_EnglishBertTask1wholeDataTrain_ResultsBestModel_EnglishSetTest_CSV_'


#Filetest results English Bert Trained in the English data
####test whole data
Task1TestFiles['BertEnglish_TrainedEnglishData_TestWholeData'] = FolderEndPathTask1['EnglishBert_enDataTrain'] + 'ModelInfereneces_EnglishBertTask1enDataTrain_ResultsBestModel_WholeSetTest_CSV_'
####test English data
Task1TestFiles['BertEnglish_TrainedEnglishData_TestEnglishData'] = FolderEndPathTask1['EnglishBert_enDataTrain'] + 'ModelInfereneces_EnglishBertTask1enDataTrain_ResultsBestModel_EnglishSetTest_CSV_'

In [ ]:
######## Spanish Bert ############
#File test results Spanish Bert Trained in the Whole data
####test whole data
Task1TestFiles['BertSpanish_TrainedwholeData_TestWholeData'] = FolderEndPathTask1['SpanishBert_wholeDataTrain'] + 'ModelInfereneces_SpanishBertTask1wholeDataTrain_ResultsBestModel_WholeSetTest_CSV_'
####test Spanish data
Task1TestFiles['BertSpanish_TrainedwholeData_TestSpanishData'] = FolderEndPathTask1['SpanishBert_wholeDataTrain'] + 'ModelInfereneces_SpanishBertTask1wholeDataTrain_ResultsBestModel_SpanishSetTest_CSV_'


####File test results SpanishBert Trained in the Spanish data
####test whole data
Task1TestFiles['BertSpanish_TrainedSpanishData_TestWholeData'] = FolderEndPathTask1['SpanishBert_esDataTrain'] + 'ModelInfereneces_SpanishBertTask1esDataTrain_ResultsBestModel_WholeSetTest_CSV_'
####test Spanish data
Task1TestFiles['BertSpanish_TrainedSpanishData_TestSpanishData'] = FolderEndPathTask1['SpanishBert_esDataTrain'] +'ModelInfereneces_SpanishBertTask1esDataTrain_ResultsBestModel_SpanishSetTest_CSV_'

In [ ]:
## print folder content
for k,v in Task1TestFiles.items():
  print(f'{k} : {v}')

BertMutiLingual_TrainedwholeData_TestWholeData : drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/MultilingualBert_wholeDataTrain/Task1/ModelInfereneces_MultilingualBertTask1wholeDataTrain_ResultsBestModel_WholeSetTest_CSV_
BertEnglish_TrainedwholeData_TestWholeData : drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/EnglishBert_wholeDataTrain/Task1/ModelInfereneces_EnglishBertTask1wholeDataTrain_ResultsBestModel_WholeSetTest_CSV_
BertEnglish_TrainedwholeData_TestEnglishData : drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/EnglishBert_wholeDataTrain/Task1/ModelInfereneces_EnglishBertTask1wholeDataTrain_ResultsBestModel_EnglishSetTest_CSV_
BertEnglish_TrainedEnglishData_TestWholeData : drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/EnglishBert_enDataTrain/Task1/ModelInfereneces_EnglishBertTask1enDataTrain_ResultsBestModel_WholeSetTest_CSV_
BertEnglish_TrainedEnglishData_Tes

# Criate Dataframes/classifiers

In [ ]:
def StandarScaler(df, columns=['0','1']):

  max = df.loc[:,columns].max().max()
  min = df.loc[:,columns].min().min()
  mean = df.loc[:,columns].mean().mean()

  for col in columns:
    df['SS'+col] = (df.loc[:,col] - mean) / (max - min)

  return df

In [ ]:
def CleanDataframe(df):
  df_clean = df.copy()
  # df_clean['Classifier'] = NameColumn
  # if drop:
  #   df_clean = df_clean.drop(columns='Label')
  df_clean = df_clean.sort_values('id').reset_index()


  return df_clean

In [ ]:
# Classifier 1 - Bert Multilingual
df_BML_TrainW_testW = pd.read_csv(Task1TestFiles['BertMutiLingual_TrainedwholeData_TestWholeData'] + '.csv', index_col=0)

StandarScaler(df_BML_TrainW_testW, ['0','1'])
df_c1 = CleanDataframe(df_BML_TrainW_testW)
df_c1.head()

,index,0,1,Inference,id,Label,SS0,SS1
0,927,2.379672,-2.225999,0,5,0,0.378390,-0.357153
1,1266,-3.029078,2.894317,1,6,1,-0.485408,0.460581
2,756,2.161592,-2.355010,0,9,0,0.343562,-0.377757
3,777,1.279254,-0.922207,0,15,0,0.202649,-0.148932
4,797,0.917143,-0.985272,0,24,0,0.144819,-0.159004


In [ ]:
# Classifier 2 - Bert ES and EN (trained only in the original data)
df_BES_TrainES_TestES = pd.read_csv(Task1TestFiles['BertSpanish_TrainedSpanishData_TestSpanishData'] + '.csv', index_col=0)
df_BEN_TrainEN_TestEN = pd.read_csv(Task1TestFiles['BertEnglish_TrainedEnglishData_TestEnglishData'] + '.csv', index_col=0)

df_BES_BEN_trainedOriginalData = pd.concat([df_BEN_TrainEN_TestEN, df_BES_TrainES_TestES])
StandarScaler(df_BES_BEN_trainedOriginalData, ['0','1'])

df_c2 = CleanDataframe(df_BES_BEN_trainedOriginalData)
df_c2.head()

,index,0,1,Inference,id,Label,SS0,SS1
0,219,1.307556,-1.286383,0,5,0,0.119113,-0.130458
1,558,-2.044864,2.404107,1,6,1,-0.203434,0.224616
2,48,1.816801,-1.648582,0,9,0,0.168109,-0.165306
3,69,-0.535696,0.909236,1,15,0,-0.058232,0.080789
4,89,1.035048,-1.389845,0,24,0,0.092894,-0.140413


In [ ]:
# Classifier 3 - Bert ES and EN (trained both slso with trainlated data)
df_BES_TrainW_TestES = pd.read_csv(Task1TestFiles['BertSpanish_TrainedwholeData_TestSpanishData'] + '.csv', index_col=0)
df_BEN_TrainW_TestEN = pd.read_csv(Task1TestFiles['BertEnglish_TrainedwholeData_TestEnglishData'] + '.csv', index_col=0)

df_BES_BEN_trainedWholeData = pd.concat([df_BEN_TrainW_TestEN ,df_BES_TrainW_TestES])

StandarScaler(df_BES_BEN_trainedWholeData, ['0','1'])
df_c3 = CleanDataframe(df_BES_BEN_trainedWholeData)
df_c3.head()

,index,0,1,Inference,id,Label,SS0,SS1
0,219,1.561726,-1.430171,0,5,0,0.197891,-0.154664
1,558,-1.704421,1.439517,1,6,1,-0.186981,0.183490
2,48,1.678124,-1.587155,0,9,0,0.211607,-0.173163
3,69,0.444956,-0.345900,0,15,0,0.066294,-0.026897
4,89,1.497078,-1.152686,0,24,0,0.190273,-0.121966


In [ ]:
# Classifier 4 - Bert EN (trained only in the original data)
df_BEN_TrainEN_TestW = pd.read_csv(Task1TestFiles['BertEnglish_TrainedEnglishData_TestWholeData'] + '.csv', index_col=0)

StandarScaler(df_BEN_TrainEN_TestW, ['0','1'])
df_c4 = CleanDataframe(df_BEN_TrainEN_TestW)
df_c4.head()

,index,0,1,Inference,id,Label,SS0,SS1
0,927,1.307556,-1.286383,0,5,0,0.233218,-0.248060
1,1266,-2.044864,2.404107,1,6,1,-0.388789,0.436672
2,756,1.816801,-1.648582,0,9,0,0.327703,-0.315263
3,777,-0.535696,0.909236,1,15,0,-0.108778,0.159314
4,797,1.035048,-1.389845,0,24,0,0.182657,-0.267257


In [ ]:
# Classifier 5 - Bert EN (trained also with trainlated data)
df_BEN_TrainW_TestW = pd.read_csv(Task1TestFiles['BertEnglish_TrainedwholeData_TestWholeData'] + '.csv', index_col=0)

StandarScaler(df_BEN_TrainW_TestW, ['0','1'])
df_c5 = CleanDataframe(df_BEN_TrainW_TestW)
df_c5.head()

,index,0,1,Inference,id,Label,SS0,SS1
0,927,1.561726,-1.430171,0,5,0,0.410684,-0.391136
1,1266,-1.704421,1.439517,1,6,1,-0.464634,0.377933
2,756,1.678124,-1.587155,0,9,0,0.441879,-0.433207
3,777,0.444956,-0.345900,0,15,0,0.111393,-0.100554
4,797,1.497078,-1.152686,0,24,0,0.393359,-0.316771


In [ ]:
# Classifier 6 - Bert ES (trained only in the original data)
df_BES_TrainES_TestW = pd.read_csv(Task1TestFiles['BertSpanish_TrainedSpanishData_TestWholeData'] + '.csv', index_col=0)

StandarScaler(df_BES_TrainES_TestW, ['0','1'])
df_c6 = CleanDataframe(df_BES_TrainES_TestW)
df_c6.head()

,index,0,1,Inference,id,Label,SS0,SS1
0,927,1.162425,-0.479003,0,5,0,0.100757,-0.057122
1,1266,-3.080168,3.180685,1,6,1,-0.307313,0.294881
2,756,3.803761,-3.011756,0,9,0,0.354811,-0.300733
3,777,0.769337,-0.623067,0,15,0,0.062948,-0.070979
4,797,2.503550,-2.240397,0,24,0,0.229751,-0.226540


In [ ]:
# Classifier 7 - Bert ES (trained also with trainlated data)
df_BES_TrainW_testW = pd.read_csv(Task1TestFiles['BertSpanish_TrainedwholeData_TestWholeData'] + '.csv', index_col=0)

StandarScaler(df_BES_TrainW_testW, ['0','1'])
df_c7 = CleanDataframe(df_BES_TrainW_testW)
df_c7.head()

,index,0,1,Inference,id,Label,SS0,SS1
0,927,1.511046,-1.787054,0,5,0,0.206958,-0.180499
1,1266,-2.979208,2.885870,1,6,1,-0.320552,0.368470
2,756,3.484716,-3.834800,0,9,0,0.438822,-0.421066
3,777,-1.692708,1.014338,1,15,0,-0.169416,0.148605
4,797,0.431626,-1.207428,0,24,0,0.080149,-0.112405


In [ ]:
## Create  dataframe with all classifier results
df_Concat = pd.concat({'c1':df_c1, 
                'c2':df_c2, 
                'c3':df_c3,
                'c4':df_c4,
                'c5':df_c5,
                'c6':df_c6,
                'c7':df_c7}, axis=1)

df_Concat.head()

c1                                ...  c7                          
  index         0         1 Inference  ...  id Label       SS0       SS1
0   927  2.379672 -2.225999         0  ...   5     0  0.206958 -0.180499
1  1266 -3.029078  2.894317         1  ...   6     1 -0.320552  0.368470
2   756  2.161592 -2.355010         0  ...   9     0  0.438822 -0.421066
3   777  1.279254 -0.922207         0  ...  15     0 -0.169416  0.148605
4   797  0.917143 -0.985272         0  ...  24     0  0.080149 -0.112405

[5 rows x 56 columns]

In [ ]:
## Creating classifier 8 - Ensemble "maioria de votos"
NumberOfClassierOnDeEmsemble = 4
NumClassifer = 8
for i in range(1, NumClassifer, 2):
  if i == 1:
    df_Concat[('c'+ str(NumClassifer), 'Inference')] = df_Concat[('c' + str(i), 'Inference')]
  else:
    df_Concat[('c'+ str(NumClassifer), 'Inference')] += df_Concat[('c' + str(i), 'Inference')]

df_Concat[('c'+ str(NumClassifer), 'Inference')] =  df_Concat[('c'+ str(NumClassifer), 'Inference')]/ NumberOfClassierOnDeEmsemble
df_Concat[('c'+ str(NumClassifer), 'Inference')] =  df_Concat[('c'+ str(NumClassifer), 'Inference')].apply(lambda x: 1 if x> 0.5 else 0)

df_Concat.head()

c1                                ...    c7                            c8
  index         0         1 Inference  ... Label       SS0       SS1 Inference
0   927  2.379672 -2.225999         0  ...     0  0.206958 -0.180499         0
1  1266 -3.029078  2.894317         1  ...     1 -0.320552  0.368470         1
2   756  2.161592 -2.355010         0  ...     0  0.438822 -0.421066         0
3   777  1.279254 -0.922207         0  ...     0 -0.169416  0.148605         0
4   797  0.917143 -0.985272         0  ...     0  0.080149 -0.112405         0

[5 rows x 57 columns]

In [ ]:
## Creating classifier 9 - Ensemble "Label with higher Unstandarazed values"
NumClassifer = 9
for i in range(1, NumClassifer-1, 2):
  if i == 1:
    df_Concat[('c'+ str(NumClassifer), '0')] = df_Concat[('c' + str(i), '0')]
    df_Concat[('c'+ str(NumClassifer), '1')] = df_Concat[('c' + str(i), '1')]
  else:
    df_Concat[('c'+ str(NumClassifer), '0')] += df_Concat[('c' + str(i), '0')]
    df_Concat[('c'+ str(NumClassifer), '1')] += df_Concat[('c' + str(i), '1')]

df_Concat[('c'+ str(NumClassifer), 'Inference')] = df_Concat[('c'+ str(NumClassifer))].idxmax(axis=1).astype(int)
df_Concat.head()

c1                                ...        c8        c9                    
  index         0         1 Inference  ... Inference         0         1 Inference
0   927  2.379672 -2.225999         0  ...         0  7.014169 -6.873395         0
1  1266 -3.029078  2.894317         1  ...         1 -9.417129  8.659221         1
2   756  2.161592 -2.355010         0  ...         0  9.002557 -9.364120         0
3   777  1.279254 -0.922207         0  ...         0  0.476457 -0.599669         0
4   797  0.917143 -0.985272         0  ...         0  4.342925 -4.498071         0

[5 rows x 60 columns]

In [ ]:
## Creating classifier 10 - Ensemble "Label with higher Standarazed values"
NumClassifer = 10
for i in range(1, NumClassifer-2, 2):
  if i == 1:
    df_Concat[('c'+ str(NumClassifer), '0')] = df_Concat[('c' + str(i), 'SS0')]
    df_Concat[('c'+ str(NumClassifer), '1')] = df_Concat[('c' + str(i), 'SS1')]
  else:
    df_Concat[('c'+ str(NumClassifer), '0')] += df_Concat[('c' + str(i), 'SS0')]
    df_Concat[('c'+ str(NumClassifer), '1')] += df_Concat[('c' + str(i), 'SS1')]

df_Concat[('c'+ str(NumClassifer), 'Inference')] = df_Concat[('c'+ str(NumClassifer))].idxmax(axis=1).astype(int)

df_Concat.head()

c1                                ...        c9       c10                    
  index         0         1 Inference  ... Inference         0         1 Inference
0   927  2.379672 -2.225999         0  ...         0  1.193923 -1.083452         0
1  1266 -3.029078  2.894317         1  ...         1 -1.457575  1.390474         1
2   756  2.161592 -2.355010         0  ...         0  1.435870 -1.405192         0
3   777  1.279254 -0.922207         0  ...         0  0.210921 -0.127779         0
4   797  0.917143 -0.985272         0  ...         0  0.808599 -0.710146         0

[5 rows x 63 columns]

In [ ]:
## Creating classifier 11 - Ensemble "maioria de votos"
NumberOfClassierOnDeEmsemble = 7
NumClassifer = 11
for i in range(1, NumClassifer-3):
  if i == 1:
    df_Concat[('c'+ str(NumClassifer), 'Inference')] = df_Concat[('c' + str(i), 'Inference')]
  else:
    df_Concat[('c'+ str(NumClassifer), 'Inference')] += df_Concat[('c' + str(i), 'Inference')]

df_Concat[('c'+ str(NumClassifer), 'Inference')] =  df_Concat[('c'+ str(NumClassifer), 'Inference')]/ NumberOfClassierOnDeEmsemble
df_Concat[('c'+ str(NumClassifer), 'Inference')] =  df_Concat[('c'+ str(NumClassifer), 'Inference')].apply(lambda x: 1 if x> 0.5 else 0)

df_Concat.head()

c1                                ...       c10                           c11
  index         0         1 Inference  ...         0         1 Inference Inference
0   927  2.379672 -2.225999         0  ...  1.193923 -1.083452         0         0
1  1266 -3.029078  2.894317         1  ... -1.457575  1.390474         1         1
2   756  2.161592 -2.355010         0  ...  1.435870 -1.405192         0         0
3   777  1.279254 -0.922207         0  ...  0.210921 -0.127779         0         0
4   797  0.917143 -0.985272         0  ...  0.808599 -0.710146         0         0

[5 rows x 64 columns]

In [ ]:
## Creating classifier 12 - Ensemble "Label with higher Unstandarazed values"
NumClassifer = 12
for i in range(1, NumClassifer-4):
  if i == 1:
    df_Concat[('c'+ str(NumClassifer), '0')] = df_Concat[('c' + str(i), '0')]
    df_Concat[('c'+ str(NumClassifer), '1')] = df_Concat[('c' + str(i), '1')]
  else:
    df_Concat[('c'+ str(NumClassifer), '0')] += df_Concat[('c' + str(i), '0')]
    df_Concat[('c'+ str(NumClassifer), '1')] += df_Concat[('c' + str(i), '1')]

df_Concat[('c'+ str(NumClassifer), 'Inference')] = df_Concat[('c'+ str(NumClassifer))].idxmax(axis=1).astype(int)
df_Concat.head()

c1                                ...       c11        c12                     
  index         0         1 Inference  ... Inference          0          1 Inference
0   927  2.379672 -2.225999         0  ...         0  10.791706  -9.925163         0
1  1266 -3.029078  2.894317         1  ...         1 -16.587024  16.648121         1
2   756  2.161592 -2.355010         0  ...         0  16.439921 -15.673039         0
3   777  1.279254 -0.922207         0  ...         0   0.174401   0.595735         1
4   797  0.917143 -0.985272         0  ...         0   8.916571  -9.518158         0

[5 rows x 67 columns]

In [ ]:
## Creating classifier 13 - Ensemble "Label with higher Standarazed values"
NumClassifer = 13
for i in range(1, NumClassifer-5):
  if i == 1:
    df_Concat[('c'+ str(NumClassifer), '0')] = df_Concat[('c' + str(i), 'SS0')]
    df_Concat[('c'+ str(NumClassifer), '1')] = df_Concat[('c' + str(i), 'SS1')]
  else:
    df_Concat[('c'+ str(NumClassifer), '0')] += df_Concat[('c' + str(i), 'SS0')]
    df_Concat[('c'+ str(NumClassifer), '1')] += df_Concat[('c' + str(i), 'SS1')]

df_Concat[('c'+ str(NumClassifer), 'Inference')] = df_Concat[('c'+ str(NumClassifer))].idxmax(axis=1).astype(int)
df_Concat.head()

c1                                ...       c12       c13                    
  index         0         1 Inference  ... Inference         0         1 Inference
0   927  2.379672 -2.225999         0  ...         0  1.647011 -1.519093         0
1  1266 -3.029078  2.894317         1  ...         1 -2.357111  2.346642         1
2   756  2.161592 -2.355010         0  ...         0  2.286493 -2.186494         0
3   777  1.279254 -0.922207         0  ...         1  0.106858  0.041345         0
4   797  0.917143 -0.985272         0  ...         0  1.313902 -1.344356         0

[5 rows x 70 columns]

In [ ]:
## calculating performance metrics
TotalNumberOfClassifiers = 13
for NumClassifer in range(1, TotalNumberOfClassifiers+1):
  Output = df_Concat[('c'+ str(NumClassifer), 'Inference')].tolist()
  if NumClassifer > 7:
    Target = df_Concat[('c1', 'Label')].tolist()
  else:
    Target = df_Concat[('c'+ str(NumClassifer), 'Label')].tolist()


  average_metrics = 'macro' if df_c7['Label'].nunique() > 2 else 'binary'
  print(f'c = {NumClassifer}')
  print(f'Accuracy : {metrics.accuracy_score(Target, Output)}')
  print(f'Recall : {metrics.recall_score(Target, Output, average = average_metrics)}')
  print(f'Precision : {metrics.precision_score(Target, Output, average = average_metrics)}')
  print(f'f1-score : {metrics.f1_score(Target, Output, average= average_metrics)}\n')

c = 1
Accuracy : 0.7612903225806451
Recall : 0.7837037037037037
Precision : 0.7388268156424581
f1-score : 0.7606038820992093

c = 2
Accuracy : 0.7741935483870968
Recall : 0.802962962962963
Precision : 0.7486187845303868
f1-score : 0.7748391708363117

c = 3
Accuracy : 0.782078853046595
Recall : 0.7777777777777778
Precision : 0.7731958762886598
f1-score : 0.775480059084195

c = 4
Accuracy : 0.7318996415770609
Recall : 0.6933333333333334
Precision : 0.7370078740157481
f1-score : 0.7145038167938932

c = 5
Accuracy : 0.7655913978494624
Recall : 0.7303703703703703
Precision : 0.7727272727272727
f1-score : 0.7509520182787509

c = 6
Accuracy : 0.7347670250896058
Recall : 0.6992592592592592
Precision : 0.7386541471048513
f1-score : 0.7184170471841705

c = 7
Accuracy : 0.753405017921147
Recall : 0.7688888888888888
Precision : 0.7340876944837341
f1-score : 0.7510853835021708

c = 8
Accuracy : 0.7842293906810036
Recall : 0.7437037037037038
Precision : 0.7968253968253968
f1-score : 0.76934865900383

In [ ]:
# Classifier with the best accuracy result for task1: C11 

# Load RealData Shared Task (No labels)

In [ ]:
# Classifier 1 - Bert Multilingual Inference Results
df_BML = pd.read_csv(FolderEndPathTask1['MultilingualBert_wholeDataTrain'] + 'ModelInferenecesMultilingualBertTask1wholeDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)

df_c1_Inf = CleanDataframe(df_BML)
df_c1_Inf.head()

,index,0,1,Inference,id
0,0,1.828211,-2.298546,0,6978
1,1,2.164775,-1.963809,0,6979
2,2,-1.256163,1.425891,1,6980
3,3,-2.509989,2.517144,1,6981
4,4,2.774074,-2.637532,0,6982


In [ ]:
# Classifier 2 - Bert ES and EN (trained only in the original data) Inference Results
df_BES_ES = pd.read_csv(FolderEndPathTask1['SpanishBert_esDataTrain'] + 'ModelInferenecesSpanishBertTask1esDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)
df_BES_ES = df_BES_ES.loc[df_BES_ES['id']>=9187]

df_BEN_EN = pd.read_csv(FolderEndPathTask1['EnglishBert_enDataTrain'] + 'ModelInferenecesEnglishBertTask1enDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)
df_BEN_EN = df_BEN_EN.loc[df_BEN_EN['id']<9187]

df_BES_BEN_OriginalData = pd.concat([df_BEN_EN, df_BES_ES])
df_c2_Inf = CleanDataframe(df_BES_BEN_OriginalData)
df_c2_Inf.head()

,index,0,1,Inference,id
0,0,1.295872,-1.186037,0,6978
1,1,0.835166,-0.908593,0,6979
2,2,-1.607499,1.858016,1,6980
3,3,-0.430777,1.007864,1,6981
4,4,2.083438,-2.189569,0,6982


In [ ]:
# Classifier 3 - Bert ES and EN (trained both slso with trainlated data) Inference Results
df_BES_W = pd.read_csv(FolderEndPathTask1['SpanishBert_wholeDataTrain'] + 'ModelInferenecesSpanishBertTask1wholeDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)
df_BES_W = df_BES_W.loc[df_BES_W['id']>=9187]

df_BEN_W = pd.read_csv(FolderEndPathTask1['EnglishBert_wholeDataTrain'] + 'ModelInferenecesEnglishBertTask1wholeDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)
df_BEN_W = df_BEN_W.loc[df_BEN_W['id']<9187]

df_BES_BEN_WholeData = pd.concat([df_BEN_W, df_BES_W])
df_c3_Inf = CleanDataframe(df_BES_BEN_WholeData)
df_c3_Inf.head()

,index,0,1,Inference,id
0,0,1.494701,-1.337020,0,6978
1,1,0.940389,-0.581290,0,6979
2,2,-1.113155,0.915252,1,6980
3,3,-0.653449,0.908423,1,6981
4,4,1.595564,-1.507158,0,6982


In [ ]:
# Classifier 4 - Bert EN (trained only in the original data) Inference Results
df_BES_ES = pd.read_csv(FolderEndPathTask1['SpanishBert_esDataTrain'] + 'ModelInferenecesSpanishBertTask1esDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)

df_c4_Inf = CleanDataframe(df_BES_ES)
df_c4_Inf.head()

,index,0,1,Inference,id
0,0,1.098298,-1.305340,0,6978
1,1,2.492234,-2.277172,0,6979
2,2,1.302778,-1.282677,0,6980
3,3,-3.016346,3.054072,1,6981
4,4,3.902636,-3.060658,0,6982


In [ ]:
# Classifier 5 - Bert EN (trained also with trainlated data) Inference Results
df_BES_W = pd.read_csv(FolderEndPathTask1['SpanishBert_wholeDataTrain'] + 'ModelInferenecesSpanishBertTask1wholeDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)

df_c5_Inf = CleanDataframe(df_BES_W)
df_c5_Inf.head()

,index,0,1,Inference,id
0,0,0.171533,-0.643674,0,6978
1,1,-0.242147,-1.063416,0,6979
2,2,-0.658588,-0.150086,1,6980
3,3,-2.858464,2.337938,1,6981
4,4,4.202213,-4.377581,0,6982


In [ ]:
# Classifier 6 - Bert ES (trained only in the original data) Inference Results
df_BEN_EN = pd.read_csv(FolderEndPathTask1['EnglishBert_enDataTrain'] + 'ModelInferenecesEnglishBertTask1enDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)

df_c6_Inf = CleanDataframe(df_BEN_EN)
df_c6_Inf.head()

,index,0,1,Inference,id
0,0,1.295872,-1.186037,0,6978
1,1,0.835166,-0.908593,0,6979
2,2,-1.607499,1.858016,1,6980
3,3,-0.430777,1.007864,1,6981
4,4,2.083438,-2.189569,0,6982


In [ ]:
# Classifier 7 - Bert ES (trained also with trainlated data) Results
df_BEN_W = pd.read_csv(FolderEndPathTask1['EnglishBert_wholeDataTrain'] + 'ModelInferenecesEnglishBertTask1wholeDataTrain_ResultsBestModel_RealData_CSV_' + '.csv', index_col=0)

df_c7_Inf = CleanDataframe(df_BEN_W)
df_c7_Inf.head()

,index,0,1,Inference,id
0,0,1.494701,-1.337020,0,6978
1,1,0.940389,-0.581290,0,6979
2,2,-1.113155,0.915252,1,6980
3,3,-0.653449,0.908423,1,6981
4,4,1.595564,-1.507158,0,6982


In [ ]:
## Create  dataframe with all classifier results
df_Concat = pd.concat({'c1':df_c1_Inf, 
                'c2':df_c2_Inf, 
                'c3':df_c3_Inf,
                'c4':df_c4_Inf,
                'c5':df_c5_Inf,
                'c6':df_c6_Inf,
                'c7':df_c7_Inf}, axis=1)

df_Concat.head()

c1                                ...        c7                          
  index         0         1 Inference  ...         0         1 Inference    id
0     0  1.828211 -2.298546         0  ...  1.494701 -1.337020         0  6978
1     1  2.164775 -1.963809         0  ...  0.940389 -0.581290         0  6979
2     2 -1.256163  1.425891         1  ... -1.113155  0.915252         1  6980
3     3 -2.509989  2.517144         1  ... -0.653449  0.908423         1  6981
4     4  2.774074 -2.637532         0  ...  1.595564 -1.507158         0  6982

[5 rows x 35 columns]

In [ ]:
## Creating classifier 11 - Ensemble "maioria de votos"
NumberOfClassierOnDeEmsemble = 7
NumClassifer = 11
for i in range(1, NumClassifer-3):
  if i == 1:
    df_Concat[('c'+ str(NumClassifer), 'Inference')] = df_Concat[('c' + str(i), 'Inference')]
  else:
    df_Concat[('c'+ str(NumClassifer), 'Inference')] += df_Concat[('c' + str(i), 'Inference')]

df_Concat[('c'+ str(NumClassifer), 'Inference')] =  df_Concat[('c'+ str(NumClassifer), 'Inference')]/ NumberOfClassierOnDeEmsemble
df_Concat[('c'+ str(NumClassifer), 'Inference')] =  df_Concat[('c'+ str(NumClassifer), 'Inference')].apply(lambda x: 1 if x> 0.5 else 0)

df_Concat.head()

c1                                ...        c7                       c11
  index         0         1 Inference  ...         1 Inference    id Inference
0     0  1.828211 -2.298546         0  ... -1.337020         0  6978         0
1     1  2.164775 -1.963809         0  ... -0.581290         0  6979         0
2     2 -1.256163  1.425891         1  ...  0.915252         1  6980         1
3     3 -2.509989  2.517144         1  ...  0.908423         1  6981         1
4     4  2.774074 -2.637532         0  ... -1.507158         0  6982         0

[5 rows x 36 columns]

In [ ]:
## add id to classifier 11 inferences
df_Concat[('c11','id')] = df_Concat.loc[:, ('c1', 'id')]
df_Concat.head()

c1                                ...        c7             c11      
  index         0         1 Inference  ... Inference    id Inference    id
0     0  1.828211 -2.298546         0  ...         0  6978         0  6978
1     1  2.164775 -1.963809         0  ...         0  6979         0  6979
2     2 -1.256163  1.425891         1  ...         1  6980         1  6980
3     3 -2.509989  2.517144         1  ...         1  6981         1  6981
4     4  2.774074 -2.637532         0  ...         0  6982         0  6982

[5 rows x 37 columns]

In [ ]:
## Path to save final results task1
Path = 'drive/MyDrive/Code/EXITS/Machine-Learning-Tweets-Classification/Bert/Results/exitis2021_AI-UPV/' 
FileName = 'task1_AI-UPV_1'

In [ ]:
## create a datafreme for final results and add column shared task
df_task1_Inferences = df_Concat[('c11')].copy()
df_task1_Inferences['sharedTask'] = 'EXIST2021'

## save a CSV tpo used in task2 inferences
df_task1_Inferences.to_csv( Path + FileName + '.csv')

##change labels in column "inference"
df_task1_Inferences['Inference'] = df_task1_Inferences['Inference'].apply(lambda x: 'non-sexist' if x == 0 else 'sexist')

df_task1_Inferences = df_task1_Inferences.loc[: ,['sharedTask', 'id', 'Inference']] 
df_task1_Inferences = df_task1_Inferences.sort_values('id')

# change ids to string
df_task1_Inferences.id = df_task1_Inferences.id.astype(str)

#visualazin data
df_task1_Inferences.head()

,sharedTask,id,Inference
0,EXIST2021,6978,non-sexist
1,EXIST2021,6979,non-sexist
2,EXIST2021,6980,sexist
3,EXIST2021,6981,sexist
4,EXIST2021,6982,non-sexist


In [ ]:
## save task1 final results to csv
df_task1_Inferences.to_csv( Path + FileName + '.csv')

In [ ]:
## add zeros to id (conf. format)
df_task1_Inferences['id'] = df_task1_Inferences['id'].apply(lambda x: '00' + x if len(x) == 4 else '0' + x)
df_task1_Inferences.head()

,sharedTask,id,Inference
0,EXIST2021,006978,non-sexist
1,EXIST2021,006979,non-sexist
2,EXIST2021,006980,sexist
3,EXIST2021,006981,sexist
4,EXIST2021,006982,non-sexist


In [ ]:
df_task1_Inferences.tail()

,sharedTask,id,Inference
4363,EXIST2021,011341,sexist
4364,EXIST2021,011342,non-sexist
4365,EXIST2021,011343,non-sexist
4366,EXIST2021,011344,sexist
4367,EXIST2021,011345,non-sexist


In [ ]:
## save task1 final results to tsc (conf. format)
df_task1_Inferences.to_csv( Path + FileName + '.tsv', header=False, sep='\t', index=False)


In [ ]:
# type(df_task1_Inferences.loc[df_task1_Inferences.id == '6978', 'id'].tolist()[0])

str